# Preprocesamiento de Datos - Retail Sales Dataset
## Felipe Lucciano Santino Di Vanni Valenzuela
### Pipeline de Machine Learning - Etapa 2/3

---

## **Objetivos del Preprocesamiento**

Este notebook es la segunda etapa del pipeline de machine learning:

1. **EDA.ipynb** COMPLETADO: Análisis exploratorio profundo
2. **Preprocessing.ipynb** (Este notebook): Preparación de datos para ML
3. **Benchmarking.ipynb**: Implementación y comparación de modelos

### **Objetivos Específicos**

- **Carga de Datos**: Importar resultados del EDA
- **Transformación de Variables**: Encoding, escalado, feature engineering
- **Preparación para ML**: División train/test, creación de pipelines
- **Validación de Calidad**: Verificación de transformaciones
- **Exportación**: Datos listos para modelado

---

## **Transformaciones Aplicadas**

1. **Encoding de Variables Categóricas**: Label encoding y one-hot encoding
2. **Escalado de Features**: StandardScaler para variables numéricas
3. **Feature Engineering**: Creación de variables derivadas
4. **Tratamiento de Outliers**: Manejo basado en insights del EDA
5. **Creación de Variable Target**: Definición del problema de clasificación
6. **División de Datos**: Train/validation/test splits estratificados

---

## **Outputs del Preprocessing**

Al finalizar este notebook tendremos:
- Datasets preparados para entrenamiento y evaluación
- Transformadores ajustados para aplicar a nuevos datos
- Documentación de todas las transformaciones aplicadas
- Validación de la calidad de los datos procesados


## **1. Configuración del Entorno de Preprocesamiento**

### **Librerías para Machine Learning**

Importamos las librerías especializadas para preprocesamiento y machine learning:

- **scikit-learn**: Transformadores, escaladores, y herramientas de ML
- **pandas/numpy**: Manipulación de datos y operaciones matemáticas
- **pickle**: Serialización de objetos y transformadores
- **joblib**: Persistencia optimizada para objetos de sklearn

### **Configuración de Reproducibilidad**

Establecemos semillas aleatorias para garantizar resultados reproducibles en todas las transformaciones que involucren aleatoriedad.


In [15]:
# Importación de librerías para preprocesamiento de datos
import pandas as pd
import numpy as np
import pickle
import joblib
from datetime import datetime
import warnings

# Librerías de scikit-learn para preprocesamiento
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Configuración de reproducibilidad
np.random.seed(42)
warnings.filterwarnings('ignore')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Entorno de preprocesamiento configurado correctamente")
print("Librerías de scikit-learn importadas")
print("Semilla aleatoria establecida para reproducibilidad")


Entorno de preprocesamiento configurado correctamente
Librerías de scikit-learn importadas
Semilla aleatoria establecida para reproducibilidad


### **Carga de Datos del EDA**

Cargamos los datos procesados en la etapa de EDA y el resumen de hallazgos para informar nuestras decisiones de preprocesamiento.

**Archivos de Entrada:**
- `retail_eda_processed.csv`: Dataset con transformaciones básicas del EDA
- `eda_summary.pkl`: Resumen de hallazgos y recomendaciones

**Verificaciones:**
- Integridad de los datos cargados
- Consistencia con hallazgos del EDA
- Validación de variables creadas


In [16]:
# Carga de datos del EDA y resumen de hallazgos

print("CARGANDO DATOS DEL EDA...")
print("=" * 50)

# Cargamos el dataset procesado en el EDA
try:
    df = pd.read_csv('../data/retail_eda_processed.csv')
    print(f"COMPLETADO: Dataset cargado exitosamente: {df.shape}")
except FileNotFoundError:
    print("ERROR: No se encontró el archivo del EDA")
    print("   Ejecute primero el notebook EDA.ipynb")
    raise

# Cargamos el resumen de hallazgos del EDA
try:
    with open('../data/eda_summary.pkl', 'rb') as f:
        hallazgosEDA = pickle.load(f)
        print(f"COMPLETADO: Resumen de EDA cargado exitosamente")
except FileNotFoundError:
    print("ADVERTENCIA: No se encontró el resumen del EDA")
    hallazgosEDA = {}

# Convertimos la columna de fecha a datetime si no está ya convertida
df['date'] = pd.to_datetime(df['date'])

# Verificamos la integridad de los datos
print(f"\nVERIFICACIÓN DE INTEGRIDAD:")
print("-" * 40)
print(f"• Dimensiones del dataset: {df.shape}")
print(f"• Período de datos: {df['date'].min().strftime('%Y-%m-%d')} a {df['date'].max().strftime('%Y-%m-%d')}")
print(f"• Valores faltantes: {df.isnull().sum().sum()}")
print(f"• Clientes únicos: {df['customer_id'].nunique()}")

# Verificamos variables creadas en el EDA
variablesEsperadas = ['year', 'month', 'day', 'dayOfWeek', 'quarter', 'rangoEtario']
variablesPresentes = [var for var in variablesEsperadas if var in df.columns]
print(f"• Variables del EDA presentes: {len(variablesPresentes)}/{len(variablesEsperadas)}")

if len(variablesPresentes) == len(variablesEsperadas):
    print("COMPLETADO: Todas las variables del EDA están presentes")
else:
    print(f"FALTANTE: Variables faltantes: {set(variablesEsperadas) - set(variablesPresentes)}")

# Mostramos información básica del dataset
print(f"\nINFORMACIÓN DEL DATASET:")
print("-" * 40)
print(f"Columnas disponibles: {list(df.columns)}")
print(f"\nPrimeras 3 filas:")
print(df.head(3))


CARGANDO DATOS DEL EDA...
COMPLETADO: Dataset cargado exitosamente: (1000, 16)
COMPLETADO: Resumen de EDA cargado exitosamente

VERIFICACIÓN DE INTEGRIDAD:
----------------------------------------
• Dimensiones del dataset: (1000, 16)
• Período de datos: 2023-01-01 a 2024-01-01
• Valores faltantes: 0
• Clientes únicos: 1000
• Variables del EDA presentes: 6/6
COMPLETADO: Todas las variables del EDA están presentes

INFORMACIÓN DEL DATASET:
----------------------------------------
Columnas disponibles: ['transaction_id', 'date', 'customer_id', 'gender', 'age', 'product_category', 'quantity', 'price_per_unit', 'total_amount', 'year', 'month', 'day', 'dayOfWeek', 'quarter', 'weekOfYear', 'rangoEtario']

Primeras 3 filas:
   transaction_id       date customer_id  gender  age product_category  quantity  price_per_unit  total_amount  year  month  day  dayOfWeek  quarter  weekOfYear   rangoEtario
0               1 2023-11-24     CUST001    Male   34           Beauty         3              50  

## **2. Definición del Problema y Variable Target**

### **Problema de Machine Learning**

Basándose en los hallazgos del EDA, definimos nuestro problema de clasificación:

**Objetivo:** Predecir categorías de ventas (Alto, Medio, Bajo) basándose en características de cliente, producto y temporales.

**Justificación:** Esta clasificación permite:
- Segmentación automática de transacciones
- Identificación de clientes de alto valor
- Optimización de estrategias de marketing
- Predicción de ingresos por categoría


In [17]:
# Creación de variable target y selección de features

print("DEFINICIÓN DEL PROBLEMA DE ML...")
print("=" * 50)

# Creamos la variable target basada en cuartiles del monto total
df['categoriaVentas'] = pd.qcut(df['total_amount'], q=3, labels=['Bajo', 'Medio', 'Alto'])

# Seleccionamos features para el modelo
featuresNumericas = ['age', 'quantity', 'price_per_unit', 'month', 'day', 'dayOfWeek', 'quarter']
featuresCategoricas = ['gender', 'product_category', 'rangoEtario']

print(f"✓ Variable target creada: 'categoriaVentas'")
print(f"✓ Features numéricas: {len(featuresNumericas)} variables")
print(f"✓ Features categóricas: {len(featuresCategoricas)} variables")

# Verificamos la distribución de la variable target
print(f"\nDISTRIBUCIÓN DE LA VARIABLE TARGET:")
print("-" * 40)
distribucionTarget = df['categoriaVentas'].value_counts()
print(distribucionTarget)
print(f"\nBalance de clases: {(distribucionTarget.min() / distribucionTarget.max() * 100):.1f}% equilibrio")

# Preparamos datos para transformación
X = df[featuresNumericas + featuresCategoricas].copy()
y = df['categoriaVentas'].copy()

print(f"\nDIMENSIONES PARA ML:")
print(f"• X (features): {X.shape}")
print(f"• y (target): {y.shape}")

# Guardamos la información de features para el próximo notebook
featuresInfo = {
    'numericas': featuresNumericas,
    'categoricas': featuresCategoricas,
    'target': 'categoriaVentas',
    'target_labels': ['Bajo', 'Medio', 'Alto']
}

print(f"✓ Problema de ML definido correctamente")


DEFINICIÓN DEL PROBLEMA DE ML...
✓ Variable target creada: 'categoriaVentas'
✓ Features numéricas: 7 variables
✓ Features categóricas: 3 variables

DISTRIBUCIÓN DE LA VARIABLE TARGET:
----------------------------------------
categoriaVentas
Medio    352
Bajo     349
Alto     299
Name: count, dtype: int64

Balance de clases: 84.9% equilibrio

DIMENSIONES PARA ML:
• X (features): (1000, 10)
• y (target): (1000,)
✓ Problema de ML definido correctamente


## **3. Implementación de ColumnTransformer y Pipeline**

### **ColumnTransformer para Transformaciones Específicas**

Utilizamos ColumnTransformer para aplicar transformaciones específicas a diferentes tipos de columnas de manera automatizada y eficiente.

### **Pipeline para Automatización**

Creamos un pipeline completo que automatiza todo el preprocesamiento y asegura la reproducibilidad del proceso.


In [18]:
# Implementación de ColumnTransformer y Pipeline para preprocesamiento automatizado

print("IMPLEMENTANDO COLUMNTRANSFORMER Y PIPELINE...")
print("=" * 60)

# Definimos los transformadores para cada tipo de columna
transformadorNumerico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

transformadorCategorico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

# Creamos el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformadorNumerico, featuresNumericas),
        ('cat', transformadorCategorico, featuresCategoricas)
    ],
    remainder='drop'
)

print(f"✓ ColumnTransformer configurado:")
print(f"  • Transformador numérico: SimpleImputer + StandardScaler")
print(f"  • Transformador categórico: SimpleImputer + OneHotEncoder")
print(f"  • Variables numéricas: {featuresNumericas}")
print(f"  • Variables categóricas: {featuresCategoricas}")

# Creamos pipeline completo de preprocesamiento
pipeline_preprocesamiento = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

print(f"✓ Pipeline de preprocesamiento creado")

# Encoding de la variable target
targetEncoder = LabelEncoder()
y_encoded = targetEncoder.fit_transform(y)

print(f"✓ Target encoder configurado")
print(f"  • Clases: {targetEncoder.classes_}")

# División estratificada de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print(f"✓ División estratificada completada:")
print(f"  • Train set: {X_train.shape}")
print(f"  • Test set: {X_test.shape}")
print(f"  • Distribución de clases balanceada")

# Aplicamos el pipeline de preprocesamiento
X_train_processed = pipeline_preprocesamiento.fit_transform(X_train)
X_test_processed = pipeline_preprocesamiento.transform(X_test)

print(f"✓ Pipeline aplicado exitosamente:")
print(f"  • Train procesado: {X_train_processed.shape}")
print(f"  • Test procesado: {X_test_processed.shape}")

# Obtenemos los nombres de las features después de la transformación
feature_names_out = []

# Features numéricas mantienen sus nombres
feature_names_out.extend(featuresNumericas)

# Features categóricas se expanden con OneHotEncoder
encoder = preprocessor.named_transformers_['cat'].named_steps['encoder']
if hasattr(encoder, 'get_feature_names_out'):
    cat_features = encoder.get_feature_names_out(featuresCategoricas)
    feature_names_out.extend(cat_features)
else:
    # Fallback para versiones anteriores de sklearn
    for i, cat in enumerate(featuresCategoricas):
        n_categories = len(encoder.categories_[i]) - 1  # -1 por drop='first'
        feature_names_out.extend([f"{cat}_{j}" for j in range(n_categories)])

print(f"✓ Feature names generados: {len(feature_names_out)} features totales")

# Verificamos la calidad de las transformaciones
print(f"\nVERIFICACIÓN DE CALIDAD DE TRANSFORMACIONES:")
print("-" * 50)
print(f"• Sin valores faltantes en train: {not np.isnan(X_train_processed).any()}")
print(f"• Sin valores faltantes en test: {not np.isnan(X_test_processed).any()}")
print(f"• Media de features numéricas ≈ 0: {np.abs(X_train_processed[:, :len(featuresNumericas)].mean(axis=0)).max() < 0.01}")
print(f"• Std de features numéricas ≈ 1: {np.abs(X_train_processed[:, :len(featuresNumericas)].std(axis=0) - 1).max() < 0.01}")
print(f"• Features categóricas binarias: {np.all(np.isin(X_train_processed[:, len(featuresNumericas):], [0, 1]))}")

# Guardamos todos los objetos necesarios para reproducibilidad
datosProcesados = {
    'X_train': X_train_processed,
    'X_test': X_test_processed,
    'y_train': y_train,
    'y_test': y_test,
    'pipeline_preprocesamiento': pipeline_preprocesamiento,
    'target_encoder': targetEncoder,
    'feature_names': feature_names_out,
    'features_info': {
        'numericas': featuresNumericas,
        'categoricas': featuresCategoricas,
        'target': 'categoriaVentas',
        'target_labels': targetEncoder.classes_
    }
}

# Guardamos usando joblib para optimizar el almacenamiento
joblib.dump(datosProcesados, '../data/datos_ml_procesados.pkl')

# Guardamos también componentes individuales para flexibilidad
joblib.dump(pipeline_preprocesamiento, '../data/preprocessed/preprocessor.joblib')
joblib.dump(targetEncoder, '../data/preprocessed/target_encoder.joblib')

# Guardamos los arrays transformados por separado para eficiencia
np.save('../data/preprocessed/X_train_transformed.npy', X_train_processed)
np.save('../data/preprocessed/X_test_transformed.npy', X_test_processed)
np.save('../data/preprocessed/y_train_encoded.npy', y_train)
np.save('../data/preprocessed/y_test_encoded.npy', y_test)

# Guardamos información de preprocesamiento
preprocessing_info = {
    'fecha_procesamiento': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'shape_original': X.shape,
    'shape_train_processed': X_train_processed.shape,
    'shape_test_processed': X_test_processed.shape,
    'feature_names': feature_names_out,
    'target_classes': targetEncoder.classes_.tolist(),
    'transformaciones_aplicadas': {
        'numericas': 'SimpleImputer(median) + StandardScaler',
        'categoricas': 'SimpleImputer(constant) + OneHotEncoder(drop_first)',
        'target': 'LabelEncoder'
    }
}

with open('../data/preprocessed/preprocessing_info.pkl', 'wb') as f:
    pickle.dump(preprocessing_info, f)

print(f"\nPREPROCESAMIENTO CON PIPELINE COMPLETADO")
print("=" * 60)
print(f"✓ Archivo principal: '../data/datos_ml_procesados.pkl'")
print(f"✓ Pipeline guardado: '../data/preprocessed/preprocessor.joblib'")
print(f"✓ Arrays procesados: '../data/preprocessed/*.npy'")
print(f"✓ Información detallada: '../data/preprocessed/preprocessing_info.pkl'")
print(f"\nLISTO PARA BENCHMARKING")
print(f"   Siguiente paso: ejecutar 'Benchmarking.ipynb'")


IMPLEMENTANDO COLUMNTRANSFORMER Y PIPELINE...
✓ ColumnTransformer configurado:
  • Transformador numérico: SimpleImputer + StandardScaler
  • Transformador categórico: SimpleImputer + OneHotEncoder
  • Variables numéricas: ['age', 'quantity', 'price_per_unit', 'month', 'day', 'dayOfWeek', 'quarter']
  • Variables categóricas: ['gender', 'product_category', 'rangoEtario']
✓ Pipeline de preprocesamiento creado
✓ Target encoder configurado
  • Clases: ['Alto' 'Bajo' 'Medio']
✓ División estratificada completada:
  • Train set: (800, 10)
  • Test set: (200, 10)
  • Distribución de clases balanceada
✓ Pipeline aplicado exitosamente:
  • Train procesado: (800, 12)
  • Test procesado: (200, 12)
✓ Feature names generados: 12 features totales

VERIFICACIÓN DE CALIDAD DE TRANSFORMACIONES:
--------------------------------------------------
• Sin valores faltantes en train: True
• Sin valores faltantes en test: True
• Media de features numéricas ≈ 0: True
• Std de features numéricas ≈ 1: True
• Fea